https://www.nature.com/articles/s41567-023-02332-9
https://github.com/ChrisWLynn/Heavy_tailed_connectivity

In [ ]:
using PyPlot, Random, Distributions, LinearAlgebra, FFTW
rc("axes.spines", top=false, right=false)

In [ ]:
threshold = 10^(-6) # Convergence threshold

In [ ]:
% Maximum number of steps:
max_steps = 10^6;

% Initialize activities:
x = x0;

% Loop until threshold is reached:
diff = 1;
count = 1;

while diff > threshold
    
    % Change in activities:
    dx = tanh(beta*(A*x + b)) - x;
    diff = max(abs(dx));
    
    % Compute new magnetizations:
    x = x + alpha*dx;